In [35]:
from tensorflow.keras.models import load_model
from numpy import genfromtxt, float as np_float
import logzero

from glocalx import GLocalX, shut_up_tensorflow
from models import Rule
import pandas as pd
import torch.nn as nn
import torch
from metadata.meta import CheckPoint

C:\Users\012709558\AppData\Local\Temp\ipykernel_21092\3551093233.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import genfromtxt, float as np_float


ModuleNotFoundError: No module named 'aggregations'

In [11]:
data0= pd.read_csv('D:/HOLDA-main/GMM_data/synthatic_data_C0.csv', header=None)
data1 = pd.read_csv('D:/HOLDA-main/GMM_data/synthatic_data_C1.csv', header=None)
data2= pd.read_csv('D:/HOLDA-main/GMM_data/synthatic_data_C2.csv', header=None)
data3 = pd.read_csv('D:/HOLDA-main/GMM_data/synthatic_data_C3.csv', header=None)

In [12]:
data0

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.067518,0.408258,0.661829,-0.061895,0.523159,-0.119963,0.713182,1.126691,-0.087939,-0.115781,0.485823,1.297351
1,0.041815,0.462968,0.501334,-0.185798,0.420244,0.169841,1.171493,0.666240,-0.026371,-0.015970,0.215262,0.873095
2,0.586180,0.392299,0.598101,0.053966,1.210092,-0.141127,0.739851,1.094166,-0.208288,-0.005515,0.599234,0.802386
3,0.756380,0.661780,1.135262,-0.179240,0.138319,0.067036,0.946233,0.973692,0.205119,0.031694,0.533219,0.962262
4,0.022663,0.377780,-0.018675,-0.160983,0.839443,-0.111338,1.597072,1.247703,0.089112,-0.029173,0.712625,0.996214
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.742451,0.489519,0.647500,0.782862,0.684651,0.994327,1.230017,-0.023522,0.112006,-0.033457,0.786265,1.123012
9996,-0.511974,0.602552,0.937057,0.514424,0.758834,0.973194,0.880446,0.219359,-0.292271,0.128567,0.033612,1.036679
9997,0.309185,0.449395,0.443633,0.377060,0.611252,0.540850,1.132455,0.162010,0.117166,-0.071151,0.306530,0.956834
9998,0.506952,0.337122,0.539794,0.892025,0.236773,0.809250,0.184534,0.751842,-0.378429,0.007458,0.349348,0.934460


In [13]:
dataset= pd.concat([data0, data1, data2, data3])

In [15]:
data = dataset.to_numpy

In [17]:
names = ['age', 'workclass', 'educational-num', 'marital-status', 'occupation',
        'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week',
        'native-country']
class_name = ["<=50K",">50K"]

In [31]:



class SimpleNet(nn.Module):
    def __init__(self, hidden_1, dropout, output, input):
        super(SimpleNet, self).__init__()

        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, output)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x


class Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, dropout, output,
                 input
                 ):

        super(Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_2, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.out(x)
        return x


class Layer3Net(nn.Module):

    def __init__(self, hidden_1, hidden_2, hidden_3,
                 dropout, output, input
                 ):

        super(Layer3Net, self).__init__()
        self.fc1 = nn.Linear(input, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_3, output)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.out(x)
        return x


def create_simple_net(hidden_1, dropout, output, input):
    net = SimpleNet(hidden_1, dropout, output, input)
    return net


def create_net(hidden_1=300, hidden_2=100, dropout=0.1, output=2,
               input=85):
    net = Net(hidden_1, hidden_2, dropout, output,
              input)
    return net

def create_layer3net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                     dropout_rate, n_classes, n_features):
    net = Layer3Net(hidden_layer_1, hidden_layer_2, hidden_layer_3,
                    dropout_rate, n_classes, n_features)
    return net


In [32]:
bbs = create_net(input = 12, hidden_1  = 256, hidden_2 = 256, dropout = 0.1, output = 2  )

In [33]:
ckpt_Ser = torch.load(open('D:/HOLDA-main/examples/experiments/not_hier/ServerSCkpt/ServerS_best.pt', 'rb'))

ModuleNotFoundError: No module named 'metadata'

In [21]:
local_explanations = Rule.from_json('D:/HOLDA-main/Rule_tree_merged.json', names=features_names)
# local_explanations_1 = Rule.from_json('D:/HOLDA-main/Rule_tree_1.json', names=features_names)
# local_explanations_2 = Rule.from_json('D:/HOLDA-main/Rule_tree_2.json', names=features_names)
# local_explanations_3 = Rule.from_json('D:/HOLDA-main/Rule_tree_3.json', names=features_names)

In [22]:
# Set log profile: INFO for normal logging, DEBUG for verbosity
logzero.loglevel(logzero.logging.INFO)
shut_up_tensorflow()

# Load black box: optional! Use black_box = None to use the dataset labels
black_box = load_model('D:/HOLDA-main/examples/experiments/not_hier/ServerSCkpt/global_model.h5')
# Load data and header
# data = genfromtxt('D:/HOLDA-main/GMM_data/dummy_dataset.csv', delimiter=',', names=True)
features_names = names
tr_set = data.view(np_float).reshape(data.shape + (-1,))

# Load local explanations
# local_explanations = Rule.from_json('data/dummy/dummy_rules.json', names=features_names)

# Create a GLocalX instance for `black_box`
glocalx = GLocalX(oracle=black_box)
# Fit the model, use batch_size=128 for larger datasets
glocalx = glocalx.fit(local_explanations, tr_set, batch_size=2, name='black_box_explanations')

# Retrieve global explanations by fidelity
alpha = 0.5
global_explanations = glocalx.rules(alpha, tr_set)
# Retrieve global explanations by fidelity percentile
alpha = 95
global_explanations = glocalx.rules(alpha, tr_set, is_percentile=True)
# Retrieve exactly `alpha` global explanations, `alpha/2` per class
alpha = 10
global_explanations = glocalx.rules(alpha, tr_set)

OSError: Unable to open file (file signature not found)

In [3]:
global_explanations

[{0: [0.5, inf], 1: [40, inf]}
 0,
 {2: [-inf, 0.5]}
 0,
 {1: [32, inf], 2: [0.5, inf]}
 1,
 {0: [0.5, inf], 1: [16, inf]}
 1]